In [1]:
import math
import re
import sys
!{sys.executable} -m pip install pyfinance
from pyfinance.options import BSM
import pandas as pd
pd.set_option("display.max_columns", None)
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy.optimize import fsolve
from scipy.optimize import minimize 

     |████████████████████████████████| 57 kB 4.5 MB/s eta 0:00:011


In [2]:
data = pd.read_excel("data/isx2010C.xls")
data = data.drop(data.columns[50], axis="columns")
data = data.rename(columns={data.columns[0]:'T', data.columns[48]:'S', data.columns[49]:'r'})
data.columns = data.columns.astype(str)
data['r'] /= 100
data['T_norm'] = data['T'] / 252
common = ['T', 'T_norm', 'S', 'r']
cols = [*common, *filter(lambda x: re.search('[0-9]+', x), data.columns)]
data = data[cols]

In [3]:
data.head()

,T,T_norm,S,r,340,345,350,355,360,365,370,375,380,385,390,395,400,405,410,415,420,425,430,435,440,445,450,455,460,465,470,475,480,485,490,495,500,505,510,515,520,525,530,535,540,545,550,555,560,565,570
0,86,0.341270,491.34,0.0011,152.20,NaN,NaN,NaN,132.60,NaN,NaN,NaN,113.20,NaN,NaN,NaN,94.55,NaN,NaN,NaN,76.45,NaN,NaN,NaN,59.25,NaN,NaN,NaN,43.60,NaN,NaN,NaN,29.80,NaN,NaN,NaN,18.65,NaN,NaN,NaN,10.45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,85,0.337302,494.35,0.0011,155.05,NaN,NaN,NaN,135.35,NaN,NaN,NaN,115.90,NaN,NaN,NaN,96.90,NaN,NaN,NaN,78.75,NaN,NaN,NaN,61.45,NaN,NaN,NaN,45.45,NaN,NaN,NaN,31.40,NaN,NaN,NaN,19.80,NaN,NaN,NaN,11.15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,84,0.333333,490.36,0.0011,152.40,NaN,NaN,NaN,132.70,NaN,NaN,NaN,113.30,NaN,NaN,NaN,94.35,NaN,NaN,NaN,76.15,NaN,NaN,NaN,59.05,NaN,NaN,NaN,43.40,NaN,NaN,NaN,29.55,NaN,NaN,NaN,18.45,NaN,NaN,NaN,10.20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,83,0.329365,486.99,0.0011,147.00,NaN,NaN,NaN,127.25,NaN,NaN,NaN,108.05,NaN,NaN,NaN,89.70,NaN,NaN,NaN,71.95,NaN,NaN,NaN,55.25,NaN,NaN,NaN,39.70,NaN,NaN,NaN,26.95,NaN,NaN,NaN,16.45,NaN,NaN,NaN,8.95,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,82,0.325397,484.11,0.0011,145.55,NaN,NaN,NaN,125.95,NaN,NaN,NaN,106.75,NaN,NaN,NaN,88.10,NaN,NaN,NaN,70.35,NaN,NaN,NaN,53.65,NaN,NaN,NaN,38.60,NaN,NaN,NaN,25.70,NaN,NaN,NaN,15.70,NaN,NaN,NaN,7.95,NaN,NaN,NaN,3.8,NaN,NaN,NaN,1525.0,NaN,NaN


In [4]:
def test(S, E, r, T, Cobs, vola):
    eps = np.finfo(float).eps
    d1 = (np.log(S / E) + (r + vola**2 / 2) * T) / (np.sqrt(vola**2 * T) + eps)
    d2 = d1 - np.sqrt(vola**2 * T)
    C = norm.cdf(d1) * S - E * np.exp(-r * T) * norm.cdf(d2)
    return C


def vola(S, E, r, T, Cobs, initial_guess=0.5, prnt=False, lr=0.01, tolerance=0.0001):
    eps = np.finfo(float).eps
    vola = initial_guess
    d = 1
    error = 1
    while(abs(error) > tolerance):
        d1 = (np.log(S / E) + (r + vola ** 2 / 2) * T) / (vola * np.sqrt(T) + eps)
        d2 = d1 - vola * np.sqrt(T)
        C = S * norm.cdf(d1) - E * np.exp(-r * T) * norm.cdf(d2)
        vDer = S * np.sqrt(T) * np.exp(-d1 ** 2 / 2) / (np.sqrt(2 * np.pi) + eps)
        d = (Cobs - C) / -vDer
        vola = vola - lr*d
        error = Cobs-C
        if prnt:
            print(f"vola: {vola}, C: {C}, error: {Cobs-C}, derivative: {vDer}, d: {d}")
    return vola

def black_scholes(vol, S, E, r, T, Cobs):
    eps = np.finfo(float).eps
    d1 = (np.log(S / E) + (r + vol ** 2 / 2) * T) / (vol * np.sqrt(T) + eps)
    d2 = d1 - vol * np.sqrt(T)
    C = S * norm.cdf(d1) - E * np.exp(-r * T) * norm.cdf(d2)
    return abs(Cobs-C)



In [5]:
#E = '0.34'
E = '340'
df_single = data[[*common, '340']]
df_single['S'] #/= 1000
df_single['340'] #/= 1000
#df_single.rename(columns={'340': '0.34'}, inplace=True)

rows_iter = df_single.iterrows()
(index, row) = next(rows_iter)
long_prev = row[E]
S = row['S']
T = row['T_norm']
r = row['r']
sigma = minimize(black_scholes, 1, args=(S, float(E), r, T, row[E]))['x'][0]
bsm = BSM(S, float(E), T, r, sigma)
delta = bsm.delta()
short_prev = delta * S

mse = 0

for index, row in rows_iter:
    long = row[E]
    short = delta * S
    dlong = long - long_prev
    dshort = short - short_prev
    mse += (dlong - dshort)**2 + 1
    long_prev = long
    short_prev = short
    
    if index % 2 == 0:
        S = row['S']
        T = row['T_norm']
        r = row['r']
        
        sigma = minimize(black_scholes, 1, args=(S, float(E), r, T, row[E]))['x'][0]
        bl_price = BSM(kind='call', S0=S, K=float(E), T=T, r=r, sigma=sigma).value() # for seeing error
        error = bl_price-row[E]
        T_val = row['T']
        print(f"T: {T_val}, {sigma=}, {error=}")
        d1 = (math.log(S/float(E)) + (r + sigma**2 / 2)*T)/(sigma * math.sqrt(T) + np.finfo(float).eps)
        delta = norm.cdf(d1)
    
mse

T: 84.0, sigma=0.30587628564063774, error=-1.4182942245695074
T: 82.0, sigma=0.35420679671591365, error=-1.4781204527025693e-07
T: 80.0, sigma=0.1984769744523866, error=-0.07492967393204708
T: 78.0, sigma=0.3039317979911047, error=-1.2990574156554544
T: 76.0, sigma=0.3382624888955499, error=-1.1061234772569151e-07
T: 74.0, sigma=0.3233727724238595, error=-2.7659808665703167e-08
T: 72.0, sigma=0.36520201557983545, error=-1.013484620671079e-07
T: 70.0, sigma=0.25792073661442305, error=-1.323201104241889e-08
T: 68.0, sigma=0.1351270444252061, error=0.09338893319193176
T: 66.0, sigma=0.13292157977312075, error=0.06232762977879247
T: 64.0, sigma=0.31263114947447146, error=-0.8990216534433841
T: 62.0, sigma=0.38603417675130186, error=-7.945220659166807e-08
T: 60.0, sigma=0.31247708216531234, error=-0.4391286604613356
T: 58.0, sigma=0.13649281080175893, error=0.1047733827540469
T: 56.0, sigma=0.35969345718279505, error=-6.279512376750063e-08
T: 54.0, sigma=0.4969761793396047, error=-1.5143598

nan

In [96]:

## Unrelated

test_row = df_single[df_single['T'] == 66]
S=test_row['S'].item()
E=0.34
r=test_row['r'].item()
T=test_row['T_norm'].item()
Cobs=test_row['0.34'].item()
test_row

KeyError: '0.34'

In [85]:
implied_vol = BSM(kind='call', S0=S, K=0.34, T=T, r=r, sigma=0.3).implied_vol(Cobs)
implied_vol

/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:151: RuntimeWarning: overflow encountered in double_scalars
  + (self.r + 0.5 * self.sigma ** 2) * self.T
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:242: RuntimeWarning: divide by zero encountered in double_scalars
  vol = vol + diff / opt.vega()
/opt/conda/lib/python3.8/site-packages/pyfinance/options.py:149: RuntimeWarning: invalid value encountered in double_scalars
  self.d1 = (


nan

In [86]:
BSM(kind='call', S0=S, K=0.34, T=T, r=r, sigma=0.17687401).value()

0.16714344277159882

In [88]:
vola(S, E, 0.0008, T, Cobs, 1.5, prnt=False)


0.23045424336670764

In [128]:
eps = np.finfo(float).eps
vol = -0.6323161324654457
d1 = (np.log(S / E) + (r + vol ** 2 / 2) * T) / (vol * np.sqrt(T) + eps)
d2 = d1 - vol * np.sqrt(T)
C = S * norm.cdf(d1) - E * np.exp(-r * T) * norm.cdf(d2)
C

-0.007312370293296316

In [212]:
mse

nan